# TASK 1


In [4]:
"""
AUTO-ONBOARD SYNC — Inovexus Automation Prototype
-------------------------------------------------
This script automates startup onboarding by:
1. Reading Typeform responses
2. Cleaning and validating data
3. Syncing data to Affinity (CRM), Notion, and Roundtable
4. Sending a welcome email through Mailchimp
"""

import requests
import json
from typing import Dict

# ----------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------
TYPEFORM_API_KEY = "YOUR_TYPEFORM_API_KEY"
AFFINITY_API_KEY = "YOUR_AFFINITY_API_KEY"
NOTION_API_KEY = "YOUR_NOTION_API_KEY"
MAILCHIMP_API_KEY = "YOUR_MAILCHIMP_API_KEY"
ROUNDTABLE_API_KEY = "YOUR_ROUNDTABLE_API_KEY"

# ----------------------------------------------------
# CORE FUNCTIONS
# ----------------------------------------------------

def clean_startup_data(data: Dict) -> Dict:
    """Standardize and sanitize startup data."""
    cleaned = {
        "company_name": data.get("company_name", "").strip().title(),
        "founder_name": data.get("founder_name", "").title(),
        "email": data.get("email", "").lower(),
        "website": data.get("website", "").lower(),
        "sector": data.get("sector", "").title(),
        "country": data.get("country", "").upper(),
        "investment_ready": data.get("investment_ready", False),
    }
    print(" Cleaned Data:", cleaned)
    return cleaned


def sync_to_affinity(data: Dict):
    """Create or update a startup record in Affinity CRM."""
    headers = {"Authorization": f"Bearer {AFFINITY_API_KEY}"}
    payload = {
        "name": data["company_name"],
        "website": data["website"],
        "custom_fields": {"Sector": data["sector"], "Country": data["country"]},
    }
    print(f" Syncing {data['company_name']} to Affinity CRM...")
    # requests.post("https://api.affinity.co/v1/organizations", headers=headers, json=payload)
    return True


def sync_to_notion(data: Dict):
    """Append a new entry to Notion database."""
    headers = {
        "Authorization": f"Bearer {NOTION_API_KEY}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }
    payload = {
        "parent": {"database_id": "YOUR_NOTION_DATABASE_ID"},
        "properties": {
            "Name": {"title": [{"text": {"content": data["company_name"]}}]},
            "Website": {"url": data["website"]},
            "Sector": {"rich_text": [{"text": {"content": data["sector"]}}]},
            "Country": {"rich_text": [{"text": {"content": data["country"]}}]},
        },
    }
    print(f" Adding {data['company_name']} to Notion database...")
    # requests.post("https://api.notion.com/v1/pages", headers=headers, json=payload)
    return True


def sync_to_roundtable(data: Dict):
    """Create SPV record in Roundtable (if startup is investment-ready)."""
    if not data["investment_ready"]:
        print(f"Skipping Roundtable (not ready): {data['company_name']}")
        return False
    headers = {"Authorization": f"Bearer {ROUNDTABLE_API_KEY}"}
    payload = {"company": data["company_name"], "status": "Pending Review"}
    print(f" Creating Roundtable SPV for {data['company_name']}...")
    # requests.post("https://api.roundtable.co/startups", headers=headers, json=payload)
    return True


def send_welcome_email(data: Dict):
    """Send personalized onboarding email via Mailchimp."""
    headers = {"Authorization": f"Bearer {MAILCHIMP_API_KEY}"}
    email_payload = {
        "email_address": data["email"],
        "status": "subscribed",
        "merge_fields": {"FNAME": data["founder_name"], "COMPANY": data["company_name"]}
    }
    print(f" Sending welcome email to {data['email']}...")
    # requests.post("https://usX.api.mailchimp.com/3.0/lists/YOUR_LIST_ID/members", headers=headers, json=email_payload)
    return True


def process_onboarding(form_response: Dict):
    """Main function to process the entire onboarding pipeline."""
    print("\nStarting onboarding workflow...")
    cleaned = clean_startup_data(form_response)
    sync_to_affinity(cleaned)
    sync_to_notion(cleaned)
    sync_to_roundtable(cleaned)
    send_welcome_email(cleaned)
    print("Onboarding complete for:", cleaned["company_name"])


# ----------------------------------------------------
# TEST WITH A MOCK RESPONSE
# ----------------------------------------------------
if __name__ == "__main__":
    mock_response = {
        "company_name": "inovexus ai",
        "founder_name": "sarah leclair",
        "email": "SARAH@INOVEXUS.AI",
        "website": "https://inovexus.ai",
        "sector": "deep tech",
        "country": "fr",
        "investment_ready": True
    }
    process_onboarding(mock_response)



Starting onboarding workflow...
 Cleaned Data: {'company_name': 'Inovexus Ai', 'founder_name': 'Sarah Leclair', 'email': 'sarah@inovexus.ai', 'website': 'https://inovexus.ai', 'sector': 'Deep Tech', 'country': 'FR', 'investment_ready': True}
 Syncing Inovexus Ai to Affinity CRM...
 Adding Inovexus Ai to Notion database...
 Creating Roundtable SPV for Inovexus Ai...
 Sending welcome email to sarah@inovexus.ai...
Onboarding complete for: Inovexus Ai
